## TFX Pipelines

#### Generate Examples

In [ ]:
# Temporary commands to unzip the zip file in Google Cloud Storage
# ! gsutil -m cp gs://text-analysis-323506/train_data/train_val.zip ./
# ! unzip train_val.zip
# ! gunzip *.csv.gz
# ! gsutil -m mv *.csv gs://text-analysis-323506/train_data/

In [ ]:
# ! pip3 install tfx==1.4.0
# ! pip install pyparsing==2.4.2

__Restart kernel__

In [93]:
import time
import os
import pickle

import tfx

import absl
import os
import tempfile
import time

import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers import TextVectorization

from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tfx.components import StatisticsGen, CsvExampleGen, SchemaGen, ExampleValidator, Transform

from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor

from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

In [2]:
tf.__version__

'2.6.2'

In [3]:
tfx.__version__

'1.4.0'

In [10]:
DATA_ROOT = 'gs://text-analysis-323506/covertype'
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')

In [11]:
PIPELINE_NAME = 'covertype-analysis'
PIPELINE_ROOT = os.path.join('/home/jupyter/', PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [12]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

### CSV Example Generator

In [13]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[        
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

In [14]:
example_gen = CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [ ]:
context.run(example_gen)

In [16]:
examples_uri = example_gen.outputs['examples'].get()[0].uri

In [17]:
tfrecord_filenames = [os.path.join(examples_uri, 'Split-train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'Split-train'))]

In [18]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

2021-12-04 10:28:24.881727: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
        print('{}: {}'.format(name, value))
    print('******')

Vertical_Distance_To_Hydrology: [101]
Slope: [9]
Wilderness_Area: [b'Commanche']
Horizontal_Distance_To_Hydrology: [648]
Horizontal_Distance_To_Fire_Points: [1871]
Horizontal_Distance_To_Roadways: [757]
Elevation: [3142]
Hillshade_9am: [223]
Aspect: [183]
Hillshade_3pm: [157]
Cover_Type: [1]
Soil_Type: [b'C7757']
Hillshade_Noon: [247]
******
Hillshade_3pm: [105]
Hillshade_9am: [245]
Aspect: [124]
Slope: [16]
Vertical_Distance_To_Hydrology: [9]
Soil_Type: [b'C2704']
Wilderness_Area: [b'Cache']
Hillshade_Noon: [227]
Elevation: [1967]
Horizontal_Distance_To_Fire_Points: [451]
Cover_Type: [2]
Horizontal_Distance_To_Hydrology: [60]
Horizontal_Distance_To_Roadways: [124]
******


2021-12-04 10:28:24.986417: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


#### Train and eval datasets have been created properly !

### Statistics Generator

In [20]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [ ]:
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### Schema Generator

In [23]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [ ]:
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

In [26]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [ ]:
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)

In [ ]:
tfdv.display_schema(schema=schema)

#### Write schema to new file

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

### Schema Importer

In [30]:
schema_importer = tfx.dsl.components.common.importer.Importer(
      source_uri=schema_dir,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer') 

In [ ]:
context.run(schema_importer)

In [ ]:
context.show(schema_importer.outputs['result'])

### ExampleValidator

In [33]:
example_validator = ExampleValidator(    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']).with_id(
          'example_validator') 

In [ ]:
context.run(example_validator)

In [ ]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "Split-train/SchemaDiff.pb")
!cat $train_anomalies_filename

In [36]:
context.show(example_validator.outputs['anomalies'])

### Transform

In [38]:
TRANSFORM_MODULE = 'preprocessing.py'

In [41]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [ ]:
context.run(transform)

In [43]:
transform.outputs['transformed_examples'].get()[0].uri

'/home/jupyter/covertype-analysis/20211204_102754/Transform/transformed_examples/7'

In [44]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['Split-train', 'Split-eval']

In [45]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'Split-train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'Split-train'))]

In [46]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
    print('{}: {}'.format(name, value))
    print('******')

Hillshade_9am_xf: [0.4025527536869049]
******
Elevation_xf: [-3.5473971366882324]
******


### Trainer

In [49]:
TRAINER_MODULE_FILE = 'model.py'

In [54]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [ ]:
context.run(trainer)

#### Tensorboard

In [56]:
logs_path = trainer.outputs['model_run'].get()[0].uri
print(logs_path)

/home/jupyter/covertype-analysis/20211204_102754/Trainer/model_run/10


### Tuner

In [57]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

context.run(tuner)

 #### Retraining with best parameters

In [64]:
hparams_importer = tfx.dsl.components.common.importer.Importer(
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=tuner.outputs['best_hyperparameters'].get()[0].uri,
    artifact_type=HyperParameters).with_id('hparams_importer') 

In [ ]:
context.run(hparams_importer)

#### Train

In [66]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    hyperparameters=hparams_importer.outputs['result'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [ ]:
context.run(trainer)

### Evaluator

In [97]:
model_resolver = Resolver(
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')

In [ ]:
context.run(model_resolver)

In [103]:
accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99})
                )

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

In [104]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)

In [105]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

In [ ]:
context.run(model_analyzer, enable_cache=False)

In [108]:
model_blessing_uri = model_analyzer.outputs['blessing'].get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Dec  4 18:34 BLESSED


In [110]:
evaluation_uri = model_analyzer.outputs['evaluation'].get()[0].uri
!ls {evaluation_uri}

attributions  eval_config.json	metrics  plots	validations


In [ ]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

In [118]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [119]:
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Wilderness_Area:Cach…

### InfraValidator

In [120]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

In [ ]:
context.run(infra_validator, enable_cache=False)

### Pusher

In [ ]:
trainer.outputs['model']

In [123]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))

In [ ]:
context.run(pusher)

In [ ]:
pusher.outputs

In [126]:
latest_pushed_model = os.path.join(SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR)))

In [130]:
_ = os.system(f"ls {latest_pushed_model}")

assets
keras_metadata.pb
saved_model.pb
variables
